In [1]:
import numpy as np
import matplotlib.pyplot as plt
from aeons.utils import *
from aeons.regress import *
from aeons.endpoint import *
from aeons.plotting import *
figsettings()

In [41]:
def get_logXfs(model, method, iterations, **kwargs):
    """Method takes in arguments (ndead, logL, nk, logZdead, **kwargs) and returns logXfs_set"""
    clear = kwargs.pop('clear', True)
    N = len(iterations)
    logXfs = np.zeros(N)
    logXfs_std = np.zeros(N)
    for i, ndead in enumerate(iterations):
        points = model.points(ndead)
        try:
            logXfs_set, message = method(points, ndead, **kwargs)
            logXfs[i] = np.mean(logXfs_set)
            logXfs_std[i] = np.std(logXfs_set)
            if clear:
                print('\r', f"Iteration {ndead} of {iterations[-1]}, {message}", end='')
            else:
                print(f"Iteration {ndead} of {iterations[-1]}, {message}")
        except:
            logXfs[i] = np.nan
            logXfs_std[i] = np.nan
            print(f"Iteration {ndead} of {iterations[-1]}, NaN")
    return logXfs, logXfs_std

In [18]:
def data(points):
    logL = points.logL.to_numpy()
    nk = points.nlive.to_numpy()
    X_mean = X_mu(nk)
    logZdead = points.logZ()
    return logL, X_mean, nk, logZdead 

In [73]:
def logXf_basic(points, ndead, Nset=25):
    logL, X_mean, nk, logZdead = data(points)
    logLd = logL[ndead:]
    logXf_set = np.zeros(Nset)
    for i in range(Nset):
        X = generate_Xs(nk)
        Xd = X[ndead:]
        theta = analytic_lm_params(logLd, Xd, d0=1)
        logXf_set[i] = logXf_formula(theta, logZdead, X_mean[ndead])
    logXf_set = logXf_set[~np.isnan(logXf_set)]
    logXf_set = reject_outliers(logXf_set)
    return logXf_set, f'{len(logXf_set)} samples'

def logXf_beta_DKL(points, ndead, Nset=25):
    logL, X_mean, nk, logZdead = data(points)
    beta_DKL = get_beta(points, ndead)
    dG = points.set_beta(beta_DKL).d_G(Nset).values
    logLd = logL[ndead:]
    logXf_set = np.zeros(Nset)
    for i in range(Nset):
        X = generate_Xs(nk)
        Xd = X[ndead:]
        d = np.random.choice(dG)
        theta = params_from_d(logLd, Xd, d)
        logXf_set[i] = logXf_formula(theta, logZdead, X_mean[ndead])
    logXf_set = logXf_set[~np.isnan(logXf_set)]
    # logXf_set = reject_outliers(logXf_set)
    return logXf_set, f"{len(logXf_set)} samples, {dG.mean():.1f}"

def logXf_beta_DKL_boot(points, ndead, Nset=25, Nresample=50):
    logL, X_mean, nk, logZdead = data(points)
    beta_DKL = get_beta(points, ndead)
    dG = points.set_beta(beta_DKL).d_G(Nset).values
    logXf_set = np.zeros(Nset)
    for i in range(Nset):
        window = np.sort(np.random.choice(np.arange(ndead, len(logL)), Nresample))
        X = generate_Xs(nk)
        logLd = logL[window]
        Xd = X[window]
        d = np.random.choice(dG)
        theta = params_from_d(logLd, Xd, d)
        logXf_set[i] = logXf_formula(theta, logZdead, X_mean[ndead])
    logXf_set = logXf_set[~np.isnan(logXf_set)]
    # logXf_set = reject_outliers(logXf_set)
    return logXf_set, f"{len(logXf_set)} samples, {dG.mean():.1f}"

In [93]:
name, samples = get_samples('other', 'gp')
model = EndModel(samples)
true_endpoint = model.true_endpoint()
true_logXf = model.true_logXf()

In [94]:
iterations = make_iterations(true_endpoint, 25)
logXfs, logXfs_std = get_logXfs(model, logXf_beta_DKL, iterations, Nset=25)

 Iteration 88235 of 88235, 25 samples, 20.65